In [1]:
import math

## Setup

In [2]:
# # 3rm
# model_size = 431080 * 4 * 8
# param_size = round(model_size / 32)
# add_p = 0.01375
# add_time_per_million = 0.0195
# mul_time_per_million = 0

# nr_layer = 4
# nr_worker = 3
# recv_ratio = 98.35 / 100

# bandwidth_dict = {}
# bandwidth_dict["s2c"] = 60.42 * 1000 * 1000
# bandwidth_dict["c2c"] = 60.42 * 1000 * 1000
# bandwidth_dict["c2w"] = 851.1 * 1000 * 1000

# time_cmd = 0.0008

# gis_real_value = 0.22
# emp_real_value = 1.7
# agg_real_value = 0.073
# dec_real_value = 0.081

In [3]:
# # 6rm
# model_size = 431080 * 4 * 8
# param_size = round(model_size / 32)
# add_time_per_million = 0.0195
# add_p = 0.01375
# mul_time_per_million = 0

# nr_layer = 4
# nr_worker = 6
# recv_ratio = 98.35 / 100

# bandwidth_dict = {}
# bandwidth_dict["s2c"] = 60.42 * 1000 * 1000
# bandwidth_dict["c2c"] = 60.42 * 1000 * 1000
# bandwidth_dict["c2w"] = 851.1 * 1000 * 1000

# time_cmd = 0.001

# gis_real_value = 0.23
# emp_real_value = 5.29
# agg_real_value = 0.392
# dec_real_value = 0.171

In [4]:
# 12rm
model_size = 431080 * 4 * 8
param_size = round(model_size / 32)
add_time_per_million = 0.0195
add_p = 0.01375
mul_time_per_million = 0

nr_layer = 4
nr_worker = 12
recv_ratio = 98.35 / 100

bandwidth_dict = {}
bandwidth_dict["s2c"] = 60.42 * 1000 * 1000
bandwidth_dict["c2c"] = 60.42 * 1000 * 1000
bandwidth_dict["c2w"] = 851.1 * 1000 * 1000

time_cmd = 0.0014

gis_real_value = 0.26
emp_real_value = 12.27
agg_real_value = 1.918
dec_real_value = 0.362

In [5]:
def ratio(pred_value, real_value):
    return pred_value / real_value

## GIS Prediction

In [6]:
def gis_prediction():
    return model_size / bandwidth_dict["s2c"] + model_size / bandwidth_dict["s2c"] * (1-recv_ratio) * (nr_worker-1)

In [7]:
gis_prediction()

0.27351676398543545

In [8]:
gis_real_value

0.26

In [9]:
ratio(gis_prediction(), gis_real_value)

1.0519875537901364

## Local Training Prediction

In [10]:
label_name = "resnet_18"

model_flops_dict = {
    "original": 2307720 * 2 * 3,
    "resnet_18": 13412720 * 2 * 3,
    "resnet_34": 23747720 * 2 * 3,
    "resnet_50": 25437720 * 2 * 3
}

lt_real_value_dict = {
    "original": 13.57,
    "resnet_18": 26.76,
    "resnet_34": 42.67,
    "resnet_50": 45.167
}

model_flops = model_flops_dict[label_name]

data_kargs = {}
data_kargs["data"] = {}
data_kargs["label"] = {}
data_kargs["data_size"] = 24754
data_kargs["batch_size"] = 64
data_kargs["data"]["shape"] = [28, 28]
data_kargs["data"]["type_bits"] = 32
data_kargs["label"]["shape"] = [1]
data_kargs["label"]["type_bits"] = 64

fb_kargs = {}
fb_kargs["mbps"] = 186.615
fb_kargs["constant"] = 0.0138

cmp_kargs = {}
cmp_kargs["gflops"] = 94.86
cmp_kargs["constant"] = 0.016

lt_real_value = lt_real_value_dict[label_name]

In [11]:
model_flops

80476320

In [12]:
_, _, pred_time = local_training_predictor(model_flops, data_kargs, fb_kargs, cmp_kargs)

NameError: name 'local_training_predictor' is not defined

In [13]:
pred_time

NameError: name 'pred_time' is not defined

In [14]:
pred_time / lt_real_value

NameError: name 'pred_time' is not defined

## EMP

In [15]:
def emp_prediction():
    time_fixed_model = (2 * model_size) / bandwidth_dict["c2c"]
    return (nr_worker - 1) * (1 + recv_ratio) * time_fixed_model + mul_time_per_million * (param_size/1000000)

In [16]:
emp_prediction()

9.962813881496192

In [17]:
emp_real_value

12.27

In [18]:
ratio(emp_prediction(), emp_real_value)

0.8119652715155821

## Aggregation prediction

In [19]:
nr_layer

4

In [20]:
(param_size/1000000) * add_time_per_million

0.00840606

In [21]:
2 * nr_layer * ((3 - 1) * 3 * time_cmd)

0.0672

In [22]:
time_cmd

0.0014

In [23]:
(0.0055 / 400000) * 1000000

0.013749999999999998

In [24]:
nr_worker

12

In [25]:
add_p

0.01375

In [26]:
p_list = [500, 20, 25000, 50, 400000, 500, 5000, 10]

In [27]:
time_cmd = 0.0014

In [28]:
agg_time = 0
for p in p_list:
    single_add_time = add_p * (p / 1000000) + time_cmd * nr_worker
    layer_time = single_add_time * (nr_worker-1)
    print(layer_time)
    agg_time += layer_time

0.184875625
0.184803025
0.18858124999999998
0.18480756249999997
0.24530000000000002
0.184875625
0.18555624999999998
0.1848015125


In [29]:
agg_time

1.5436008499999998

In [30]:
nr_worker

12

In [31]:
time_cmd * nr_worker

0.0168

In [32]:
add_p * (5000 / 1000000)

6.875e-05

In [33]:
add_p * (400000 / 1000000) + time_cmd * nr_worker

0.0223

In [34]:
add_p * (param_size / 1000000) * (nr_worker - 1)

0.06520085

In [35]:
2 * nr_layer * ((nr_worker - 1) * (nr_worker * time_cmd))

1.4784

In [36]:
add_time_per_million

0.0195

In [ ]:
def agg_prediction():
    return 2 * nr_layer * ((nr_worker - 1) * (nr_worker * time_cmd)) + (nr_worker - 1) * nr_worker *  (param_size/1000000) * add_p

In [37]:
# def agg_prediction():
#     return 2 * nr_layer * ((nr_worker - 1) * (nr_worker * time_cmd)) + (nr_worker - 1) *  (param_size/1000000) * add_p

In [38]:
agg_prediction()

1.54360085

In [39]:
agg_real_value

1.918

In [40]:
ratio(agg_prediction(), agg_real_value)

0.8047971063607925

In [41]:
ratio(agg_time, agg_real_value)

0.8047971063607924

## Decryption prediction

In [33]:
def dec_prediction():
    return nr_worker * (model_size * 2 / bandwidth_dict["c2w"])

In [34]:
dec_prediction()

5.208851039830807

In [35]:
dec_real_value

10.2

In [36]:
ratio(dec_prediction(), dec_real_value)

0.5106716705716478